In [ ]:
import re
import json
from typing import Dict, Optional, List, Tuple
import pymorphy3

# Инициализируем морфологический анализатор
morph = pymorphy3.MorphAnalyzer()


def normalize_text(text: str) -> str:
    """
    Нормализует текст, приводя все слова к начальной форме.
    
    Args:
        text: Исходный текст
        
    Returns:
        Нормализованный текст с словами в начальной форме
    """
    words = text.split()
    normalized_words = []
    
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word)
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            normalized_words.append(parsed.normal_form)
        else:
            normalized_words.append(word)
    
    return ' '.join(normalized_words)


def generate_word_forms(word: str) -> set:
    """
    Генерирует все возможные формы слова (падежи, числа) через pymorphy3.
    
    Args:
        word: Слово в начальной форме
        
    Returns:
        Множество всех форм слова
    """
    parsed = morph.parse(word)[0]
    forms = {word.lower()}
    
    # Генерируем все формы слова
    for form in parsed.lexeme:
        forms.add(form.word.lower())
    
    return forms


def create_flexible_pattern(alias: str) -> str:
    """
    Создает гибкий паттерн для алиаса с учетом склонений.
    
    Args:
        alias: Алиас закона в нижнем регистре
        
    Returns:
        Регулярное выражение (строка) с учетом склонений
    """
    # Разбиваем алиас на токены (слова и не-слова)
    tokens = re.findall(r'[а-яёa-z]+|[^а-яёa-z]+', alias, re.IGNORECASE)
    
    pattern_parts = []
    
    for token in tokens:
        if re.match(r'^[а-яёa-z]+$', token, re.IGNORECASE):
            # Это слово - генерируем все формы
            word_forms = generate_word_forms(token)
            
            # Если слово длинное (>3 символов) и имеет разные формы, создаем альтернативы
            if len(word_forms) > 1 and len(token) > 3:
                # Сортируем формы по длине (от длинных к коротким) для более точного матчинга
                sorted_forms = sorted(word_forms, key=len, reverse=True)
                escaped_forms = [re.escape(form) for form in sorted_forms]
                pattern_parts.append(f"(?:{'|'.join(escaped_forms)})")
            else:
                # Короткое слово или нет форм - оставляем как есть
                pattern_parts.append(re.escape(token))
        else:
            # Это не слово (пробелы, знаки препинания) - делаем гибким
            # Пробелы могут быть опциональными или множественными
            if token.strip() == '':
                pattern_parts.append(r'\s+')
            else:
                pattern_parts.append(re.escape(token))
    
    pattern = ''.join(pattern_parts)
    
    # Добавляем границы слов в начале и конце, если нужно
    if alias and re.match(r'[\wа-яёА-ЯЁ]', alias[0]):
        pattern = r'\b' + pattern
    if alias and re.match(r'[\wа-яёА-ЯЁ]', alias[-1]):
        pattern = pattern + r'\b'
    
    return pattern


def load_law_aliases_with_morphology():
    """
    Загружает law_aliases.json и создает индекс с нормализованными формами.
    ОПТИМИЗАЦИЯ: предкомпилирует все регулярные выражения один раз.
    Учитывает склонения слов в алиасах.
    
    Returns:
        Tuple[Dict, List]: (нормализованный_алиас -> [(оригинал, law_id)], все алиасы отсортированные)
    """
    with open('law_aliases.json', 'r', encoding='utf-8') as f:
        law_aliases = json.load(f)
    
    normalized_index = {}
    all_aliases = []
    
    for law_id, aliases in law_aliases.items():
        for alias in aliases:
            alias_lower = alias.lower()
            normalized = normalize_text(alias_lower)
            
            if normalized not in normalized_index:
                normalized_index[normalized] = []
            normalized_index[normalized].append((alias_lower, law_id))
            
            # ОПТИМИЗАЦИЯ: создаем гибкий паттерн с учетом склонений
            flexible_pattern_str = create_flexible_pattern(alias_lower)
            compiled_pattern = re.compile(flexible_pattern_str, re.IGNORECASE)
            
            # Также создаем точный паттерн для быстрого матчинга (без склонений)
            escaped_alias = re.escape(alias_lower)
            if re.match(r'[\w]', alias_lower):
                exact_pattern_str = r'\b' + escaped_alias
            else:
                exact_pattern_str = escaped_alias
            if re.search(r'[\w]$', alias_lower):
                exact_pattern_str = exact_pattern_str + r'\b'
            
            exact_compiled_pattern = re.compile(exact_pattern_str, re.IGNORECASE)
            
            all_aliases.append({
                'original': alias_lower,
                'normalized': normalized,
                'law_id': law_id,
                'length': len(alias_lower),
                'word_count': len(alias_lower.split()),
                'compiled_pattern': compiled_pattern,  # Гибкий паттерн с учетом склонений
                'exact_pattern': exact_compiled_pattern  # Точный паттерн для быстрого поиска
            })
    
    all_aliases.sort(key=lambda x: x['length'], reverse=True)
    
    return normalized_index, all_aliases


def find_law_in_text(text: str, normalized_index: Dict, all_aliases: List[Dict]) -> Optional[str]:
    """
    Ищет упоминание кодекса в тексте с учетом склонений.
    
    Args:
        text: Текст для поиска
        normalized_index: Индекс нормализованных форм
        all_aliases: Список всех алиасов
        
    Returns:
        law_id или None
    """
    text_lower = text.lower()
    
    # Извлекаем номера документов из текста (например, №474, №201-рп)
    numbers_in_text = re.findall(r'№\s*(\d+(?:[-./]\S*)?)', text)
    
    # Извлекаем тип документа из текста (Указ, Распоряжение, Постановление и т.д.)
    doc_type_in_text = None
    doc_type_patterns = {
        'указ': r'указ[аеуыои]?\b',
        'распоряжен': r'распоряжен[иеюя]+\b',
        'постановлен': r'постановлен[иеюя]+\b',
        'приказ': r'приказ[аеуыои]?\b',
        'закон': r'закон[аеуыои]?\b'
    }
    
    for base_type, pattern in doc_type_patterns.items():
        if re.search(pattern, text_lower):
            doc_type_in_text = base_type
            break
    
    # Если в тексте нет номера, собираем кандидатов по ключевым словам
    best_match_without_number = None
    best_match_score = 0
    
    # Сначала пробуем прямое совпадение (быстрее и точнее)
    # ОПТИМИЗАЦИЯ: используем предкомпилированные регулярные выражения
    for alias_data in all_aliases:
        alias_lower = alias_data['original']
        
        # Если в тексте есть номер документа, проверяем совместимость с алиасом
        if numbers_in_text:
            # Извлекаем номера из алиаса
            alias_numbers = re.findall(r'№\s*(\d+(?:[-./]\S*)?)', alias_lower)
            
            # Если у алиаса есть номер, он должен совпадать с номером в тексте
            if alias_numbers:
                # Нормализуем номера (убираем точки и другие знаки препинания в конце)
                normalized_text_numbers = [num.rstrip('.,;:!?') for num in numbers_in_text]
                normalized_alias_numbers = [num.rstrip('.,;:!?') for num in alias_numbers]
                has_matching_number = any(alias_num in normalized_text_numbers for alias_num in normalized_alias_numbers)
                if not has_matching_number:
                    # Пропускаем этот алиас, так как номер не совпадает
                    continue
                else:
                    # Если номер совпадает и тип документа совпадает,
                    # проверяем наличие ключевых слов (более гибко, с учетом склонений)
                    if doc_type_in_text and doc_type_in_text in alias_lower:
                        # Извлекаем ключевые слова из алиаса (слова длиннее 3 символов, буквенные)
                        alias_words = re.findall(r'\b[а-яёa-z]{4,}\b', alias_lower)
                        if alias_words:
                            # Проверяем, что большинство ключевых слов присутствуют в тексте
                            matching_count = sum(1 for word in alias_words if word in text_lower)
                            # Если хотя бы 60% ключевых слов совпадают, считаем это совпадением
                            if matching_count >= len(alias_words) * 0.6:
                                return alias_data['law_id']
        
        # Если в тексте есть тип документа, проверяем совместимость с алиасом
        if doc_type_in_text:
            # Проверяем, что алиас содержит тот же тип документа
            if doc_type_in_text not in alias_lower:
                # Пропускаем этот алиас, так как тип документа не совпадает
                continue
            
            # Если в тексте НЕТ номера, собираем кандидатов по ключевым словам (для учета склонений)
            if not numbers_in_text:
                alias_words = re.findall(r'\b[а-яёa-z]{4,}\b', alias_lower)
                if alias_words:
                    matching_count = sum(1 for word in alias_words if word in text_lower)
                    match_ratio = matching_count / len(alias_words)
                    # Сохраняем лучший результат (с максимальным процентом совпадения)
                    if match_ratio > best_match_score and match_ratio >= 0.7:
                        best_match_score = match_ratio
                        best_match_without_number = alias_data['law_id']
        
        # Сначала пробуем точное совпадение (быстрее)
        if alias_data['exact_pattern'].search(text_lower):
            return alias_data['law_id']
        
        # Если точного совпадения нет, пробуем гибкое (с учетом склонений)
        if alias_data['compiled_pattern'].search(text_lower):
            return alias_data['law_id']
    
    # Если нашли подходящий алиас по ключевым словам (для случая без номера в тексте)
    if best_match_without_number:
        return best_match_without_number
    
    # Если прямого совпадения нет, пробуем с нормализацией
    # Разбиваем текст на фразы (последовательности слов)
    word_sequences = re.finditer(r'[а-яёА-ЯЁ\w\s]+', text_lower)
    
    for seq_match in word_sequences:
        sequence = seq_match.group()
        words = sequence.split()
        
        max_window = min(10, len(words))
        
        for window_size in range(max_window, 0, -1):
            for i in range(len(words) - window_size + 1):
                window = words[i:i + window_size]
                phrase = ' '.join(window)
                
                normalized_phrase = normalize_text(phrase)
                
                if normalized_phrase in normalized_index:
                    # Дополнительная проверка для коротких алиасов
                    matches = normalized_index[normalized_phrase]
                    
                    for original_alias, law_id in matches:
                        # Если алиас короткий (как "НК", "ГК"), требуем точного совпадения
                        if len(original_alias) <= 3:
                            # Проверяем, что фраза в тексте выглядит как аббревиатура
                            if re.search(r'\b' + re.escape(phrase.upper()) + r'\b', text.upper()):
                                return law_id
                        else:
                            return law_id
    
    return None


# Загружаем данные один раз
NORMALIZED_INDEX, ALL_ALIASES = load_law_aliases_with_morphology()


def parse_legal_reference_v2(text: str) -> List[Dict[str, Optional[str]]]:
    """
    Парсит юридический текст и извлекает все упоминания статей, пунктов и подпунктов.
    Версия с поддержкой склонений через pymorphy3 и множественных ссылок.
    
    Args:
        text: Текст с упоминаниями статей, пунктов и/или подпунктов
        
    Returns:
        Список словарей с полями law_id, article, point_article, subpoint_article
    """
    results = []
    
    text_lower = text.lower().strip()
    text_stripped = text.strip()
    
    # Поиск law_id с учетом склонений
    law_id = find_law_in_text(text_lower, NORMALIZED_INDEX, ALL_ALIASES)
    
    # Паттерны для поиска статей с перечислениями
    # Ищем конструкции типа: "ст. 929, 681.14 и 1988" или "статья 100, 200 и 300"
    # Поддерживаем все склонения слова "статья": статья, статьи, статье, статью, статьёй, статьей, статей, статьям, статьями, статьях
    # Важно: "статей" (gen pl) = "стат" + "ей" (без ь), остальные формы = "стать" + окончание
    article_list_patterns = [
        r'стат(?:ь(?:ями|ях|ям|ёй|ей|[яиею])|ей)\s+((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'ст\.?\s*((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    ]
    
    # Паттерны для поиска пунктов с перечислениями
    # Разделитель: запятая (опционально с союзом) или просто союз
    point_list_patterns = [
        r'(?<!под)пункт[аеуыои]?\s+((?:(?:\d+[а-яА-Я]?|[а-яА-Я])(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'(?<![а-яА-Я])п\.?\s+((?:(?:\d+[а-яА-Я]?|[а-яА-Я])(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
    ]
    
    # Паттерны для поиска подпунктов с перечислениями
    # Подпункты могут быть: буква (я), буква+цифры (я1), или просто цифры (26)
    # Важно: одиночная буква не должна захватываться, если за ней идёт точка (п. = пункт, а не подпункт)
    # Разделитель: запятая (опционально с союзом) или просто союз
    subpoint_list_patterns = [
        r'подпункт[аеуыои]?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'подп\.?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'пп\.?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
    ]
    
    # Функция для разбора перечисления
    def parse_enumeration(enum_str: str, pattern: str) -> List[str]:
        """Разбивает строку перечисления на отдельные элементы, сохраняя регистр"""
        # Заменяем союзы на запятые для единообразия (case-insensitive)
        enum_str = re.sub(r'\s+и\s+', ',', enum_str, flags=re.IGNORECASE)
        enum_str = re.sub(r'\s+или\s+', ',', enum_str, flags=re.IGNORECASE)
        # Разбиваем по запятым
        items = [item.strip() for item in enum_str.split(',')]
        # Фильтруем пустые и оставляем только те, что соответствуют паттерну
        return [item for item in items if item and re.match(pattern, item, re.IGNORECASE)]
    
    # Поиск всех статей
    articles_found = []
    for pattern in article_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            articles = parse_enumeration(enum_str, r'^\d+(?:[.-]\d+)*$')
            articles_found.extend(articles)
    
    # Поиск всех пунктов
    points_found = []
    for pattern in point_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            points = parse_enumeration(enum_str, r'^(?:\d+[а-яА-Я]?|[а-яА-Я])$')
            # Фильтруем "п" - это сокращение для слова "пункт", а не номер пункта
            points = [p for p in points if p.lower() != 'п']
            points_found.extend(points)
    
    # Поиск всех подпунктов
    subpoints_found = []
    for pattern in subpoint_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            # Подпункты: буква (я), буква+цифры (я1), или просто цифры (26)
            subpoints = parse_enumeration(enum_str, r'^(?:[а-яА-Я]\d*|\d+)$')
            # Фильтруем "п" и "подп" - это сокращения для слова "пункт/подпункт", а не номера
            subpoints = [s for s in subpoints if s.lower() not in ['п', 'подп', 'пп']]
            subpoints_found.extend(subpoints)
    
    # Конвертируем law_id из строки в число
    law_id_int = int(law_id) if law_id is not None else None
    
    # Формируем результаты на основе найденных элементов
    # Логика: создаем запись для самого детального уровня перечисления
    # Если есть несколько подпунктов -> для каждого подпункта
    # Иначе, если есть несколько пунктов -> для каждого пункта
    # Иначе, если есть несколько статей -> для каждой статьи
    
    if subpoints_found and len(subpoints_found) > 1:
        # Есть перечисление подпунктов - создаем запись для каждого
        for subpoint in subpoints_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoint
            }
            results.append(result)
    elif points_found and len(points_found) > 1:
        # Есть перечисление пунктов - создаем запись для каждого
        for point in points_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": point,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found and len(articles_found) > 1:
        # Есть перечисление статей - создаем запись для каждой
        for article in articles_found:
            result = {
                "law_id": law_id_int,
                "article": article,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found or points_found or subpoints_found:
        # Единичное упоминание
        result = {
            "law_id": law_id_int,
            "article": articles_found[0] if articles_found else None,
            "point_article": points_found[0] if points_found else None,
            "subpoint_article": subpoints_found[0] if subpoints_found else None
        }
        results.append(result)
    else:
        # Если ничего не найдено, возвращаем пустую запись
        results.append({
            "law_id": law_id_int,
            "article": None,
            "point_article": None,
            "subpoint_article": None
        })
    
    return results




In [25]:
# Загрузка и выполнение тестов из demo_test_cases.json

def run_tests():
    """
    Загружает тесты из demo_test_cases.json и прогоняет их через parse_legal_reference_v2
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    print("=" * 80)
    print("ТЕСТИРОВАНИЕ")
    print("=" * 80)
    
    total_tests = 0
    passed_tests = 0
    failed_tests = 0
    
    for idx, test_data in enumerate(test_cases, 1):
        text = test_data['text']
        expected_results = test_data['test_cases']
        
        # Получаем результаты от функции
        actual_results = parse_legal_reference_v2(text)
        
        print(f"\nТЕСТ {idx}:")
        print(f"Текст: '{text[:100]}...'")
        print()
        
        # Сравниваем результаты
        is_passed = compare_results(expected_results, actual_results)
        
        total_tests += 1
        if is_passed:
            passed_tests += 1
            print("✅ ТЕСТ ПРОЙДЕН")
        else:
            failed_tests += 1
            print("❌ ТЕСТ НЕ ПРОЙДЕН")
        
        print(f"Найдено ссылок: {len(actual_results)}")
        print("Ожидаемые результаты:")
        print(json.dumps(expected_results, indent=2, ensure_ascii=False))
        print("Полученные результаты:")
        print(json.dumps(actual_results, indent=2, ensure_ascii=False))
        print("-" * 80)
    
    print("\n" + "=" * 80)
    print(f"ИТОГО: {passed_tests}/{total_tests} тестов пройдено")
    print(f"Успешно: {passed_tests}, Провалено: {failed_tests}")
    print("=" * 80)


def compare_results(expected, actual):
    """
    Сравнивает ожидаемые и фактические результаты
    """
    if len(expected) != len(actual):
        return False
    
    # Создаем копии для сортировки
    expected_sorted = sorted(expected, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    actual_sorted = sorted(actual, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    for exp, act in zip(expected_sorted, actual_sorted):
        if (exp.get('law_id') != act.get('law_id') or
            exp.get('article') != act.get('article') or
            exp.get('point_article') != act.get('point_article') or
            exp.get('subpoint_article') != act.get('subpoint_article')):
            return False
    
    return True


def show_test_details(test_num):
    """
    Показывает детали конкретного теста
    
    Args:
        test_num: Номер теста (1-based)
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    if test_num < 1 or test_num > len(test_cases):
        print(f"Ошибка: тест {test_num} не существует. Доступны тесты 1-{len(test_cases)}")
        return
    
    test_data = test_cases[test_num - 1]
    text = test_data['text']
    expected_results = test_data['test_cases']
    
    print(f"ТЕСТ {test_num}")
    print("=" * 80)
    print(f"Текст:\n{text}\n")
    print("-" * 80)
    
    actual_results = parse_legal_reference_v2(text)
    
    print(f"\nОжидается найти: {len(expected_results)} ссылок")
    print(f"Найдено: {len(actual_results)} ссылок")
    print()
    
    print("ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:")
    for i, exp in enumerate(expected_results, 1):
        print(f"{i}. law_id={exp.get('law_id')}, article={exp.get('article')}, "
              f"point={exp.get('point_article')}, subpoint={exp.get('subpoint_article')}")
    
    print("\nПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:")
    for i, act in enumerate(actual_results, 1):
        print(f"{i}. law_id={act.get('law_id')}, article={act.get('article')}, "
              f"point={act.get('point_article')}, subpoint={act.get('subpoint_article')}")
    
    print("\n" + "=" * 80)
    
    # Сравнение
    is_passed = compare_results(expected_results, actual_results)
    if is_passed:
        print("✅ ТЕСТ ПРОЙДЕН")
    else:
        print("❌ ТЕСТ НЕ ПРОЙДЕН")
        print("\nРАЗЛИЧИЯ:")
        show_differences(expected_results, actual_results)


def show_differences(expected, actual):
    """
    Показывает различия между ожидаемыми и фактическими результатами
    """
    if len(expected) != len(actual):
        print(f"- Количество ссылок: ожидалось {len(expected)}, получено {len(actual)}")
    
    # Находим отличия
    for i, (exp, act) in enumerate(zip(expected, actual), 1):
        differences = []
        if exp.get('law_id') != act.get('law_id'):
            differences.append(f"law_id: {exp.get('law_id')} -> {act.get('law_id')}")
        if exp.get('article') != act.get('article'):
            differences.append(f"article: {exp.get('article')} -> {act.get('article')}")
        if exp.get('point_article') != act.get('point_article'):
            differences.append(f"point: {exp.get('point_article')} -> {act.get('point_article')}")
        if exp.get('subpoint_article') != act.get('subpoint_article'):
            differences.append(f"subpoint: {exp.get('subpoint_article')} -> {act.get('subpoint_article')}")
        
        if differences:
            print(f"Ссылка {i}: {', '.join(differences)}")


# Запуск всех тестов
run_tests()


ТЕСТИРОВАНИЕ

ТЕСТ 1:
Текст: 'Во время проверки воздушного судна была обнаружена необходимость уточнения некоторых процедур. В соо...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 1
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
Полученные результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
--------------------------------------------------------------------------------

ТЕСТ 2:
Текст: 'В ходе заседания представитель компании-перевозчика указал на соблюдение всех требований безопасност...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 5
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "я"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "26"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article

In [28]:

show_test_details(14)


ТЕСТ 14
Текст:
Главный бухгалтер компании представил отчет о финансовом состоянии, ссылаясь на статьи 1836, 1670.10, 802, 270, 1351 и 930 Положения по ведению бухгалтерского учета и бухгалтерской отчетности в Российской Федерации. Он отметил, что соблюдение этих норм критично для обеспечения прозрачности и достоверности финансовой информации. В связи с этим компания планирует провести внутренний аудит на следующей неделе.

--------------------------------------------------------------------------------

Ожидается найти: 6 ссылок
Найдено: 6 ссылок

ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:
1. law_id=1036, article=1836, point=None, subpoint=None
2. law_id=1036, article=1670.10, point=None, subpoint=None
3. law_id=1036, article=802, point=None, subpoint=None
4. law_id=1036, article=270, point=None, subpoint=None
5. law_id=1036, article=1351, point=None, subpoint=None
6. law_id=1036, article=930, point=None, subpoint=None

ПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:
1. law_id=None, article=1836, point=None, subpoint=None
2. law_i

In [31]:
text = "Главный бухгалтер компании представил отчет о финансовом состоянии, ссылаясь на статьи 1836, 1670.10, 802, 270, 1351 и 930 Положения по ведению бухгалтерского учета и бухгалтерской отчетности в Российской Федерации. Он отметил, что соблюдение этих норм критично для обеспечения прозрачности и достоверности финансовой информации. В связи с этим компания планирует провести внутренний аудит на следующей неделе."
parse_legal_reference_v2(text)

[{'law_id': None,
  'article': '1836',
  'point_article': None,
  'subpoint_article': None},
 {'law_id': None,
  'article': '1670.10',
  'point_article': None,
  'subpoint_article': None},
 {'law_id': None,
  'article': '802',
  'point_article': None,
  'subpoint_article': None},
 {'law_id': None,
  'article': '270',
  'point_article': None,
  'subpoint_article': None},
 {'law_id': None,
  'article': '1351',
  'point_article': None,
  'subpoint_article': None},
 {'law_id': None,
  'article': '930',
  'point_article': None,
  'subpoint_article': None}]